In [ ]:
# Funcao get_ticker_list, ja implementada em functions.py

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=chrome_options)

driver.get('https://fundamentus.com.br/resultado.php')

local_tabela = '/html/body/div[1]/div[2]/table'
tabela = driver.find_element('xpath', local_tabela)
html_tabela = tabela.get_attribute('outerHTML')
tabela = pd.read_html(str(html_tabela))[0]  # FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
driver.quit()
tickers = tabela["Papel"].sort_values()
tickers.to_csv('data/tickers.csv', index=False)
tickers
#, options=chrome_options

In [ ]:
from b3fileparser import b3parser
import pandas as pd

dados_b3 = b3parser.read_b3_file('C:/Users/mauri/Projetos/Gayatri/GayatriPy/data/COTAHIST_A2023.TXT')
dados_b3.to_csv('C:/Users/mauri/Projetos/Gayatri/GayatriPy/data/dadosb3.csv')
dados_b3

In [37]:
import pandas as pd
tickers = pd.read_csv('C:/Users/mauri/Projetos/Gayatri/GayatriPy/data/tickers.csv')
tickers["Papel"]

0       AALR3
1       ABCB3
2       ABCB4
3       ABEV3
4       ABYA3
        ...  
981     WLMM4
982     WMBY3
983    WSON33
984     YDUQ3
985     ZAMP3
Name: Papel, Length: 986, dtype: object

In [62]:
papeis = tickers.head(5)
papeis["Papel"]

0    AALR3
1    ABCB3
2    ABCB4
3    ABEV3
4    ABYA3
Name: Papel, dtype: object

In [82]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import date, datetime, timedelta
from io import StringIO
import pandas as pd
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=chrome_options)

stocks = pd.DataFrame(columns=["Ticker", "Nome", "Setor", "Subsetor", "Última Cotação"])

for ticker in papeis["Papel"]:
    driver.get(f'https://www.fundamentus.com.br/detalhes.php?papel={ticker}')
    tabela = driver.find_element('xpath', '/html/body/div[1]/div[2]/table[1]')   
    html_tabela = tabela.get_attribute('outerHTML')
    tabela = pd.read_html(StringIO(str(html_tabela)))[0]
    nome = tabela.iloc[2, 1]
    lastpricedate = str(tabela.iloc[1, 3])
    setor = tabela.iloc[3, 1]
    subsetor = tabela.iloc[4, 1]
    stocks = pd.concat([stocks, pd.DataFrame({"Ticker": [ticker], "Nome": [nome], "Setor": [setor], "Subsetor": [subsetor], "Última Cotação": [lastpricedate]})], ignore_index=True)

stocks = stocks.dropna()
stocks.to_csv('C:/Users/mauri/Projetos/Gayatri/GayatriPy/data/Stocks.csv')
stocks

,Ticker,Nome,Setor,Subsetor,Última Cotação
0,AALR3,ALLIAR ON NM,Serv.Méd.Hospit. Análises e Diagnósticos,Serv.Méd.Hospit. Análises e Diagnósticos,11/04/2024
1,ABCB3,ABC Brasil ON N2,Intermediários Financeiros,Bancos,30/12/1899
2,ABCB4,ABC Brasil PN N2,Intermediários Financeiros,Bancos,11/04/2024
3,ABEV3,AMBEV S/A ON,Bebidas,Cervejas e Refrigerantes,11/04/2024
4,ABYA3,ABYARA ON NM,Construção Civil,Incorporações,11/02/2010


In [88]:
today = datetime.now()